In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
def addFeatures(df):
    i = 0
    l = len(df.columns)
    while i < l:
        j = i
        while j < l:
            string = str(df.columns[i]) + "*" + str(df.columns[j])
            df[string] = df[df.columns[i]] * df[df.columns[j]]
            k = j
            while k < l:
                string = str(df.columns[i]) + "*" + str(df.columns[j]) + "*" + str(df.columns[k])
                df[string] = df[df.columns[i]] * df[df.columns[j]] * df[df.columns[k]]
                k += 1
            j += 1
        i += 1
    return df

In [3]:
def data_preprocessing(x):
    deck = {'A' : 1, 'B' : 2, 'C' : 3, 'D' : 4, 'E' : 5, 'F' : 6, 'G' : 7, 'U' : 8}
    x.Cabin = x.Cabin.fillna('U0')
    x['Deck'] = x['Cabin'].map(lambda i: re.compile("([a-zA-Z]+)").search(i).group())
    x['Deck'] = x['Deck'].map(deck)
    x['Deck'] = x.Deck.fillna(0)
    x['Deck'] = x['Deck'].astype(int)
    del x['Cabin']
    
    mean = x.Age.mean()
    std = x.Age.std()
    is_null = x.Age.isnull().sum()
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    copy = x.Age.copy()
    copy[np.isnan(copy)] = rand_age
    x.Age = copy
    x.Age = x.Age.astype(int)
    x['Age'] = x.Age.fillna(0)
    
    x.Embarked = x.Embarked.fillna('S')
    x.Fare = x.Fare.astype(int)
    
    titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    x['Title'] = x.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    x.Title = x.Title.replace('Mlle', 'Miss')
    x.Title = x.Title.replace('Ms', 'Miss')
    x.Title = x.Title.replace('Mme', 'Mrs')
    
    l_to_replace = []
    for i in np.array(x.Title):
        if i not in titles:
            l_to_replace.append(i)
            
    x.Title = x.Title.replace(l_to_replace, 'Rare')
    x.Title = x.Title.map(titles)
    x.Title = x.Title.fillna(0)
    del x['Name']
    genders = {'male' : 0, 'female' : 1}
    x.Sex = x.Sex.map(genders)
    del x['Ticket']
    ports = {"S": 0, "C": 1, "Q": 2}
    x.Embarked = x.Embarked.map(ports)
    
    return x

In [4]:
data = pd.read_csv('train.csv')
x, y = data.iloc[:, :-1], data.iloc[:, -1]
# del x['Name']
# del x['Ticket']
# del x['Cabin']
# x = pd.get_dummies(x)
# x = x.fillna(0)
x = data_preprocessing(x)
x = addFeatures(x)
x = np.array(x)
scaler = StandardScaler()
x = scaler.fit_transform(x)

c:\program files\python36\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
c:\program files\python36\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
clf = LogisticRegression(solver='liblinear')
clf.fit(x, y)
clf.score(x, y)

0.8592814371257484

In [6]:
x_test = pd.read_csv('test.csv')
# del x_test['Name']
# del x_test['Ticket']
# del x_test['Cabin']
# x_test = pd.get_dummies(x_test)
# x_test = x_test.fillna(0)
x_test = data_preprocessing(x_test)
x_test = addFeatures(x_test)
x_test = np.array(x_test)
x_test = scaler.transform(x_test)

c:\program files\python36\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
y_pred = clf.predict(x_test)

In [8]:
np.savetxt('y_test.csv', y_pred, delimiter=',') 